## Desenvolvimento do modelo de predição de preço

Exploração inicial de modelos de base

In [13]:
import pandas as pd
from dagshub.data_engine import datasources
import mlflow
import dagshub
from sklearn.model_selection import train_test_split
import mlflow.sklearn
import mlflow.catboost
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
import mlflow.models.signature
from mlflow.models import infer_signature
from catboost import CatBoostRegressor

In [14]:
ds = datasources.get_datasource("pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl","processed")

In [15]:
ds.all().dataframe

Output()

,path,datapoint_id,dagshub_download_url,media type,size
0,laptop-price-brl-processed.csv,102408450,https://dagshub.com/api/v1/repos/pedromonnt/fi...,text/plain,52559


In [16]:
res = ds.head()

for dp in res:
    dataset_url = dp.download_url

Output()

In [17]:
dataset_url

'https://dagshub.com/api/v1/repos/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl/raw/main/data/processed/laptop-price-brl-processed.csv'

In [18]:
df = pd.read_csv(dataset_url)
df.head()

,ram_gb,ssd,hdd,graphic_card_gb,warranty,price,brand_asus,brand_dell,brand_hp,brand_lenovo,...,ram_type_other,os_other,os_windows,os_bit_32-bit,os_bit_64-bit,weight_casual,weight_gaming,weight_thinnlight,touchscreen_0,touchscreen_1
0,4,0,1024,0,0,2321,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0
1,4,0,1024,0,0,2613,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
2,4,0,1024,0,0,2680,0,0,0,1,...,0,0,1,0,1,1,0,0,1,0
3,8,512,0,2,0,4689,1,0,0,0,...,0,0,1,1,0,1,0,0,1,0
4,4,0,512,0,0,1808,1,0,0,0,...,0,0,1,0,1,1,0,0,1,0


## Desenvolvimento e experimentos de modelos

In [19]:
dagshub.init(repo_owner="pedromonnt", repo_name="fiap-ds-mlops-10dtsr-laptop-pricing-brl", mlflow = True)

Initialized MLflow to track repo "pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl"

Repository pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl initialized!

In [20]:
mlflow.autolog()

2025/07/09 18:51:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for lightgbm.
2025/07/09 18:51:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/09 18:51:27 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [21]:
features = list(df.columns)
features.remove("price")

In [22]:
features

['ram_gb',
 'ssd',
 'hdd',
 'graphic_card_gb',
 'warranty',
 'brand_asus',
 'brand_dell',
 'brand_hp',
 'brand_lenovo',
 'brand_other',
 'processor_brand_amd',
 'processor_brand_intel',
 'processor_brand_m1',
 'processor_name_core i3',
 'processor_name_core i5',
 'processor_name_core i7',
 'processor_name_other',
 'processor_name_ryzen 5',
 'processor_name_ryzen 7',
 'ram_type_ddr4',
 'ram_type_other',
 'os_other',
 'os_windows',
 'os_bit_32-bit',
 'os_bit_64-bit',
 'weight_casual',
 'weight_gaming',
 'weight_thinnlight',
 'touchscreen_0',
 'touchscreen_1']

In [23]:
X = df[features]

In [24]:
len(features)

30

In [25]:
y = df["price"]
y

0       2321
1       2613
2       2680
3       4689
4       1808
       ...  
775     9111
776     9714
777    10049
778     9580
779     3852
Name: price, Length: 780, dtype: int64

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [28]:
def evaluate_and_log_model(kind, model_name, model, X_test, y_test):
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)
    r2 = r2_score(y_test, predictions)
    mape = mean_absolute_percentage_error(y_test, predictions)
    
    mlflow.log_metric("MSE", mse)
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("R2", r2)
    mlflow.log_metric("MAPE", mape)

    signature = infer_signature(X_test, predictions)

    if kind == "catboost":
        mlflow.catboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "xgboost":
        mlflow.xgboost.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    elif kind == "lightgbm":
        mlflow.lightgbm.log_model(model, model_name, signature=signature, input_example=X_test[:5])
    else:
        mlflow.sklearn.log_model(model, model_name, signature=signature, input_example=X_test[:5])

    print(f"Model {model_name} logged with MSE: {mse}, MAE: {mae}, R2: {r2}, MAPE: {mape}")

### Experimento com Ridge Regression

In [29]:
with mlflow.start_run(run_name = "Ridge Regression"):
    param_grid = {
        'alpha': [0.1, 1.0, 10.0, 100.0],
        'fit_intercept': [True, False]
    }

    ridge = Ridge()

    grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=make_scorer(mean_absolute_percentage_error, greater_is_better = False))
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    mlflow.log_param("best_alpha", grid_search.best_params_['alpha'])
    mlflow.log_param("best_fit_intercept", grid_search.best_params_['fit_intercept'])

    evaluate_and_log_model("sklearn", "ridge_regression", best_model, X_test, y_test)

2025/07/09 19:46:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/07/09 19:46:12 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\

🏃 View run magnificent-snipe-42 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/a57b4127b9364fd3b322289a94923896
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run classy-loon-851 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/c8a50945fa38472a93f0e2aa4c6a88d3
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run exultant-wren-142 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/8f8b578c9d8641b69eab51b79e891570
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
🏃 View run upset-wasp-711 at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/e43f86c3e33a4eb3965b76b8720

c:\Users\pedro\Downloads\fiap-ds-mlops-10dtsr-laptop-pricing-brl\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


Model ridge_regression logged with MSE: 3024089.113899016, MAE: 1137.7871730265826, R2: 0.6421034856259351, MAPE: 0.22065408855367982
🏃 View run Ridge Regression at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0/runs/f81cea20ab854bfcb7c1aae20b4648e4
🧪 View experiment at: https://dagshub.com/pedromonnt/fiap-ds-mlops-10dtsr-laptop-pricing-brl.mlflow/#/experiments/0
